# Calculate lattice constant and cohesive energy

## Write the Input File

In [46]:
%%writefile lat_const_coh_en.in
# LAMMPS INPUT SCRIPT
# Calculate the lattice constant and cohesive energy of an fcc structure

# INITIALIZATION
clear 
units metal 
dimension 3 
boundary p p p 
atom_style atomic 
atom_modify map array
variable element string "Cu" # change this to the desired element
log log_${element}.lammps # Changes the log file name

# ATOM DEFINITION
lattice fcc 4.079 orient x 1 0 0 orient y 0 1 0 orient z 0 0 1  
region box block 0 1 0 1 0 1 units lattice
create_box 1 box
create_atoms 1 box
replicate 1 1 1

# DEFINE INTERATOMIC POTENTIAL
pair_style eam/alloy 
pair_coeff * * CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy.txt ${element}
neighbor 2.0 bin 
neigh_modify delay 10 check yes 
 
# DEFINE COMPUTES 
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 

# MINIMIZATION
reset_timestep 0 
fix 1 all box/relax iso 0.0 vmax 0.001
thermo 10 
thermo_style custom step pe lx ly lz press c_eatoms 
min_style cg 
minimize 1e-25 1e-25 5000 10000 

variable natoms equal "count(all)" 
variable teng equal "c_eatoms"
variable length equal "lx"
variable ecoh equal "v_teng/v_natoms"

print "Total energy (eV) = ${teng};"
print "Number of atoms = ${natoms};"
print "Lattice constant (Angstoms) = ${length};"
print "Cohesive energy (eV) = ${ecoh};"

# OUTPUT THE FINAL RELAXED STRUCTURE
write_data relaxed_${element}.data

print "All done!"

Overwriting lat_const_coh_en.in


## Run LAMMPS

In [47]:
!lmp < lat_const_coh_en.in

LAMMPS (29 Aug 2024 - Development - 2995cb76ae)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Lattice spacing in x,y,z = 4.079 4.079 4.079
Created orthogonal box = (0 0 0) to (4.079 4.079 4.079)
  1 by 1 by 1 MPI processor grid
Created 4 atoms
  using lattice units in orthogonal box = (0 0 0) to (4.079 4.079 4.079)
  create_atoms CPU = 0.001 seconds
Replication is creating a 1x1x1 = 1 times larger system...
  orthogonal box = (0 0 0) to (4.079 4.079 4.079)
  1 by 1 by 1 MPI processor grid
  4 atoms
  replicate CPU = 0.001 seconds
Reading eam/alloy potential file CuAgAuNiPdPtAlPbFeMoTaWMgCoTiZr_Zhou04.eam.alloy.txt with DATE: 2018-03-30
Switching to 'neigh_modify every 1 delay 0 check yes' setting during minimization
Neighbor list info ...
  update: every = 1 steps, delay = 0 steps, check = y

In [48]:
# Move the log lammps files and relaxed structure files to a folder called relaxed_structures
!mkdir -p relaxed_structures
!mv log_*.lammps relaxed_*.data relaxed_structures
